In [11]:
!pip install qiskit qiskit-aer

In [12]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np

def run_anonymous_bit_sim(sender_bit):
    # 1. Setup 4 Qubits (Alice=0, Bob=1, Charlie=2, David=3)
    qc = QuantumCircuit(4, 4)

    # 2. Create the "Magic Coin" (GHZ State)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    qc.barrier()

    # 3. Alice's Turn (The Z-Gate flip if sending a '1')
    if sender_bit == 1:
        qc.z(0)

    qc.barrier()

    # 4. The Measurement Prep (Everyone applies H to look at the 'flip')
    qc.h([0, 1, 2, 3])

    # 5. Measure
    qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

    # Run the simulation
    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(transpile(qc, backend), shots=1)
    result = job.result().get_counts()

    # Get the binary string (e.g., '1011')
    outcome_str = list(result.keys())[0]
    # Convert string to list of integers: [1, 0, 1, 1]
    bits = [int(b) for b in outcome_str]

    # Calculate Parity (XOR sum)
    parity = sum(bits) % 2
    return bits, parity

# --- TEST IT ---
my_sent_bit = 1
measurements, received_parity = run_anonymous_bit_sim(my_sent_bit)

print(f"Alice sent: {my_sent_bit}")
print(f"Node Measurements (A,B,C,D): {measurements}")
print(f"Final Calculated Parity: {received_parity}")

Alice sent: 1
Node Measurements (A,B,C,D): [0, 0, 1, 0]
Final Calculated Parity: 1


In [13]:
# Function to convert a character to 8 binary bits
def char_to_bits(char):
    return [int(b) for b in format(ord(char), '08b')]

# The message Alice wants to send anonymously
secret_message = 'a'
bits_to_send = char_to_bits(secret_message)

print(f"Original Message: {secret_message}")
print(f"Bits to transmit: {bits_to_send}\n")

received_bits = []

# THE LOOP: Running the Quantum Protocol 8 times
for i, bit in enumerate(bits_to_send):
    # Run our 'Digital Twin' simulation for each bit
    _, parity_result = run_anonymous_bit_sim(bit)
    received_bits.append(parity_result)
    print(f"Round {i+1}: Sent {bit} -> Received Parity {parity_result}")

# Final Reconstruction (Using the code you ran earlier!)
byte_string = "".join(str(b) for b in received_bits)
decoded_char = chr(int(byte_string, 2))

print(f"\nFinal Reconstructed Message: '{decoded_char}'")

Original Message: a
Bits to transmit: [0, 1, 1, 0, 0, 0, 0, 1]

Round 1: Sent 0 -> Received Parity 0
Round 2: Sent 1 -> Received Parity 1
Round 3: Sent 1 -> Received Parity 1
Round 4: Sent 0 -> Received Parity 0
Round 5: Sent 0 -> Received Parity 0
Round 6: Sent 0 -> Received Parity 0
Round 7: Sent 0 -> Received Parity 0
Round 8: Sent 1 -> Received Parity 1

Final Reconstructed Message: 'a'
